# The Weyl Scalar $\psi_4$

### ***Citations:*** This module is designed to replicate the results from the state-of-the-art [WeylScal4 diagnostic module](https://bitbucket.org/einsteintoolkit/einsteinanalysis/src/master/WeylScal4/) in the Einstein Toolkit, but building upon the generic reference metric formalism of [Ruchlin, Etienne, and Baumgarte](https://arxiv.org/abs/1712.07658).

### WeylScal4 itself is built upon the prescription of [Baker, Campanelli, and Lousto. PRD 65, 044001 (2002)](https://arxiv.org/abs/gr-qc/0104063) (henceforth, the "BCL paper"). which we follow below.

The Weyl scalar $\psi_4$, is a quantity used to extract the gravitational wave strain $h_+$ and $h_{\times}$ directly from numerical relativity simulations. 

$\psi_4$ is defined in terms of the gravitational wave strain via

$$\psi_4 = \ddot{h}_+ - i \ddot{h}_{\times}.$$ 

This module draws heavily on work already done in [BSSN_RHSs.py](BSSN/BSSN_RHSs.py). In general, this mostly provides conformal (barred) metric quantities.

More details on these quantities are documented in [the tutorial covering BSSN in curvilinear coordinates](Tutorial-BSSNCurvilinear.ipynb). We also use reference metric quantities through the module [reference_metric.py](reference_metric.py); the tutorial for that is available [here](Tutorial-Reference_Metric.ipynb).

In [1]:
# Step 1: import all needed modules from NRPy+:
import NRPy_param_funcs as par
import indexedexp as ixp
import grid as gri
import finite_difference as fin
import reference_metric as rfm

#rfm.reference_metric() # Already called in BSSN_RHSs.py
from outputC import *
import BSSN.BSSN_RHSs as bssn
import sympy as sp


In [2]:
# Step 2: Initialize WeylScalar parameters
thismodule = __name__
# Current option: Approx_QuasiKinnersley = choice made in Baker, Campanelli, and Lousto. PRD 65, 044001 (2002)
par.initialize_param(par.glb_param("char", thismodule, "TetradChoice", "Approx_QuasiKinnersley"))

### Defining the Levi-Civita Symbol

Next we declare a function that defines the rank-3 Levi-Civita symbol, $\epsilon_{ijk}$, applying the algorithm in the [Wikipedia article on the Levi-Civita symbol](https://en.wikipedia.org/wiki/Levi-Civita_symbol), but exploiting properties of computer integer arithmetic as outlined [here](https://codegolf.stackexchange.com/questions/160359/levi-civita-symbol). 

In short, the Levi-Civita symbol $\epsilon_{ijk}$ is set to 1 for even permutations of the indices, -1 for odd permutations, or zero if any indices are repeated.

In [3]:
# Step 3: Define the rank-3 version of the Levi-Civita symbol. Amongst
#         other uses, this is needed for the construction of the approximate 
#         quasi-Kinnersley tetrad.
def define_LeviCivitaSymbol_rank3(DIM=-1):
    if DIM == -1:
        DIM = par.parval_from_str("DIM")

    LeviCivitaSymbol = ixp.zerorank3()

    for i in range(DIM):
        for j in range(DIM):
            for k in range(DIM):
                # From https://codegolf.stackexchange.com/questions/160359/levi-civita-symbol :
                LeviCivitaSymbol[i][j][k] = (i - j) * (j - k) * (k - i) / 2
    return LeviCivitaSymbol

## Defining the Approximate Quasi-Kinnersley Tetrad

To define the Weyl scalars, first a tetrad must be chosen. Below, for compatibility with the [WeylScal4 diagnostic module](https://bitbucket.org/einsteintoolkit/einsteinanalysis/src/master/WeylScal4/), we implement the approximate  quasi-Kinnersley tetrad of the BCL paper.

We begin with the vectors given in eqs. 5.6 and 5.7 of the BCL paper,
\begin{align}
    v_1^a &= [-y,x,0] \\
    v_2^a &= [x,y,z] \\
    v_3^a &= {\rm det}(g)^{1/2} g^{ad} \epsilon_{dbc} v_1^b v_2^c,
\end{align}
and carry out the Gram-Schmidt orthonormalization process. Note that these vectors are initially orthogonal to each other; one is in the $\phi$ direction, one is in $r$, and the third is the cross product of the first two. The vectors $w_i^a$ are placeholders in the code; the final product of the orthonormalization is the vectors $e_i^a$. So,
\begin{align}
e_1^a &= \frac{v_1^a}{\omega_{11}} \\
e_2^a &= \frac{v_2^a - \omega_{12} e_1^a}{\omega_{22}} \\
e_3^a &= \frac{v_3^a - \omega_{13} e_1^a - \omega_{23} e_2^a}{\omega_{33}}, \\
\end{align}
where $\omega_{ij} = v_i^a v_j^b \bar{\gamma}_{ab}$ needs to be updated between steps (to save resources, we can get away with only calculating components as needed), and uses $e_i^a$ instead of $v_i^a$ if it has been calculated. For $\bar{\gamma}_{ab}$, we use the tensor calculated by BSSN_RHSs.py.

Once we have orthogonal, normalized vectors, we can contruct the tetrad itself, again drawing on eqs. 5.6. We can draw on SymPy's built-in tools for complex numbers to build the complex vectors $m^a$ and $\bar{m}^a$:
\begin{align}
    l^a &= \frac{1}{\sqrt{2}} e_2^a \\
    n^a &= \frac{-1}{\sqrt{2}} e_2^a \\
    m^a &= \frac{1}{\sqrt{2}} (e_3^a + i e_1^a) \\
    \bar{m}^a &= \frac{1}{\sqrt{2}} (e_3^a - i e_1^a)
\end{align}

We will also assume that $n^0 = \frac{1}{\sqrt{2}}$ and that $m^0 = \bar{m}^0 = 0$. This last assumption in particular will significantly reduce the terms needed to find $\psi_4$.

In [4]:
# Step 1: Call BSSN_RHSs. This module computes many different quantities related to the metric,
#         many of which will be essential here. We must first change to our desired coordinate
#         system, however.
par.set_parval_from_str("reference_metric::CoordSystem","Cartesian")
bssn.BSSN_RHSs()

# Step 2a: Set spatial dimension (must be 3 for BSSN)
DIM = 3
par.set_parval_from_str("grid::DIM",DIM)

# Step 2b: Set the coordinate system to Cartesian
x = rfm.xxCart[0]
y = rfm.xxCart[1]
z = rfm.xxCart[2]

# Step 2c: Set which tetrad is used; at the moment, only one supported option
TetradChoice = par.parval_from_str("TetradChoice")

if TetradChoice == "Approx_QuasiKinnersley":
    # Eqs 5.6 in https://arxiv.org/pdf/gr-qc/0104063.pdf
    xmoved = x# - xorig
    ymoved = y# - yorig
    zmoved = z# - zorig

    # Step 3a: Choose 3 orthogonal vectors. Here, we choose one in the azimuthal 
    #          direction, one in the radial direction, and the cross product of the two. 
    # Eqs 5.7
    v1U = ixp.zerorank1()
    v2U = ixp.zerorank1()
    v3U = ixp.zerorank1()
    v1U[0] = -ymoved
    v1U[1] = xmoved# + offset
    v1U[2] = 0
    v2U[0] = xmoved# + offset
    v2U[1] = ymoved
    v2U[2] = zmoved
    LeviCivitaSymbol_rank3 = define_LeviCivitaSymbol_rank3()
    for a in range(DIM):
        for b in range(DIM):
            for c in range(DIM):
                for d in range(DIM):
                    v3U[a] += sp.sqrt(bssn.detgammabar) * bssn.gammabarUU[a][d] * LeviCivitaSymbol_rank3[d][b][c] * v1U[b] *v2U[c]

    # Step 3b: Gram-Schmidt orthonormalization of the vectors.
    # The w_i^a vectors here are used to temporarily hold values on the way to the final vectors e_i^a

    w1U = ixp.zerorank1()
    for a in range(DIM):
        w1U[a] = v1U[a]
    omega11 = 0
    for a in range(DIM):
        for b in range(DIM):
            omega11 += w1U[a] * w1U[b] * bssn.gammabarDD[a][b]
    e1U = ixp.zerorank1()
    for a in range(DIM):
        e1U[a] = w1U[a] / sp.sqrt(omega11)

    omega12 = 0
    for a in range(DIM):
        for b in range(DIM):
            omega12 += e1U[a] * v1U[b] * bssn.gammabarDD[a][b]
    w2U = ixp.zerorank1()
    for a in range(DIM):
        w2U[a] = v2U[a] - omega12*e1U[a]
    omega22 = 0
    for a in range(DIM):
        for b in range(DIM):
            omega22 += w2U[a] * w2U[b] *bssn.gammabarDD[a][b]
    e2U = ixp.zerorank1()
    for a in range(DIM):
        e2U[a] = w2U[a] / sp.sqrt(omega22)

    omega13 = 0
    for a in range(DIM):
        for b in range(DIM):
            omega13 += e1U[a] * v3U[b] * bssn.gammabarDD[a][b]
    omega23 = 0
    for a in range(DIM):
        for b in range(DIM):
            omega23 += e2U[a] * v3U[b] * bssn.gammabarDD[a][b]
    w3U = ixp.zerorank1()
    for a in range(DIM):
        w3U[a] = v3U[a] - omega13*e1U[a] - omega23*e2U[a]
    omega33 = 0
    for a in range(DIM):
        for b in range(DIM):
            omega33 += w3U[a] * w3U[b] * bssn.gammabarDD[a][b]
    e3U = ixp.zerorank1()
    for a in range(DIM):
        e3U[a] = w3U[a] / sp.sqrt(omega33)
        
    # Step 3c: Construct the tetrad itself.
    # Eqs. 5.6
    isqrt2 = 1/sp.sqrt(2)
    ltetU = ixp.zerorank1()
    ntetU = ixp.zerorank1()
    mtetU = ixp.zerorank1()
    mtetbarU = ixp.zerorank1()
    for i in range(DIM):
        ltetU[i] = isqrt2 * e2U[i]
        ntetU[i] = -isqrt2 * e2U[i]
        mtetU[i] = isqrt2 * (e3U[i] + sp.I*e1U[i])
        mtetbarU[i] = isqrt2 * (e3U[i] + sp.I*e1U[i])
    nn = isqrt2

else:
    print("Error: TetradChoice == "+par.parval_from_str("TetradChoice")+" unsupported!")
    exit(1)

## Building the Riemann and extrinsic curvature tensors
Now that we have the tetrad in place, we can contract it with the Weyl tensor to obtain the Weyl scalars. Naturally, we must first construct the Weyl tensor to do that. We will first build the Riemann curvature tensor,
\begin{align}
\bar{R}_{abcd} = \frac{1}{2} (\bar{\gamma}_{ad,cb}+\bar{\gamma}_{bc,da}-\bar{\gamma}_{ac,bd}-\bar{\gamma}_{bd,ac}) + \bar{\gamma}_{je}\bar{\Gamma}^{j}_{bc}\bar{\Gamma}^{e}_{ad} - \bar{\gamma}_{je}\bar{\Gamma}^{j}_{bd}\bar{\Gamma}^{e}_{ac}
\end{align}
since several terms in our expression for $\psi_4$ are contractions of this tensor.
To do this, we need second derivatives of the metric tensor, $\bar{\gamma}_{ab,cd}$. Recall from Tutorial-BSSNCurvilinear that 
\begin{align}
\bar{\gamma}_{ij,k} &= \partial_k \bar{\gamma}_{ij} \\
&= \partial_k \left(h_{ij} \text{ReDD[i][j]} + \hat{\gamma}_{ij}\right) \\
&= h_{ij,k} \text{ReDD[i][j]} + h_{ij} \text{ReDDdD[i][j][k]} + \hat{\gamma}_{ij,k},
\end{align}
where $h_{ij}$ and its derivatives are drawn from BSSN_RHSs.py. It then follows from the familiar chain and product rules of differentiation that 
\begin{align}
\bar{\gamma}_{ij,kl} =& \partial_l \bar{\gamma}_{ij,k} \\
=& \partial_l (h_{ij,k} \text{ReDD[i][j]} + h_{ij} \text{ReDDdD[i][j][k]} + \hat{\gamma}_{ij,k}) \\
=& h_{ij,kl} \text{ReDD[i][j]} + h_{ij,k} \text{ReDDdD[i][j][l]} \\
&+ h_{ij,l} \text{ReDDdD[i][j][k]} + h_{ij} \text{ReDDdDD[i][j][k][l]} \\
&+ \hat{\gamma}_{ij,kl}.
\end{align}

We also need the extrinsic curvature tensor $K_{ij}$; it can be shown that 
\begin{align}
K_{ij} &= e^{4 \phi} (\bar{A}_{ij} + \frac{1}{3} \bar{\gamma}_{ij} K),
\end{align}
where $e^{4 \phi}$ is the inverse square of the conformal factor, $\bar{A}_{ij}$ is the conformal, trace-free extrinsic curvature, and $K = \bar{\gamma}^{ij} K_{ij}$ is the trace of the extrinsic curvature. All of these tensors can be pulled directly from BSSN_RHSs.py.

In [5]:
# Step 4a: Declare and construct the second derivative of the metric.
gammabarDD_dDD = ixp.zerorank4()
for i in range(DIM):
    for j in range(DIM):
        for k in range(DIM):
            for l in range(DIM):
                gammabarDD_dDD[i][j][k][l] = bssn.hDD_dDD[i][j][k][l]*rfm.ReDD[i][j] + \
                                             bssn.hDD_dD[i][j][k]*rfm.ReDDdD[i][j][l] + \
                                             bssn.hDD_dD[i][j][l]*rfm.ReDDdD[i][j][k] + \
                                             bssn.hDD[i][j]*rfm.ReDDdDD[i][j][k][l] + \
                                             rfm.ghatDDdDD[i][j][k][l]
# Step 4b: Declare and construct the barred Riemann curvature tensor:
RiemannbarDDDD = ixp.zerorank4()
for a in range(DIM):
    for b in range(DIM):
        for c in range(DIM):
            for d in range(DIM):
                RiemannbarDDDD[a][b][c][d] = (gammabarDD_dDD[a][d][c][b] + \
                                              gammabarDD_dDD[b][c][d][a] - \
                                              gammabarDD_dDD[a][c][b][d] - \
                                              gammabarDD_dDD[b][d][a][c]) / 2
                for e in range(DIM):
                    for j in range(DIM):
                        RiemannbarDDDD[a][b][c][d] +=  bssn.gammabarDD[j][e] * bssn.GammabarUDD[j][b][c] * bssn.GammabarUDD[e][a][d] - \
                                                       bssn.gammabarDD[j][e] * bssn.GammabarUDD[j][b][d] * bssn.GammabarUDD[e][a][c]

                        
# Step 4c: We also need the extrinsic curvature tensor $K_{ij}$. This can be built from quantities from BSSN_RHSs
extrinsicKDD = ixp.zerorank2()
for i in range(DIM):
    for j in range(DIM):
        extrinsicKDD[i][j] = (bssn.AbarDD[i][j] + sp.Rational(1,3)*bssn.gammabarDD[i][j]*bssn.trK)/bssn.exp_m4phi

# Putting it all together: calculating $\psi_4$
We do not not need to explicitly build the Weyl tensor itself, because the BCL paper shows that, for the Weyl tensor $C_{ijkl}$,
\begin{align}
\psi_4 =&  C_{ijkl} n^i \bar{m}^j n^k \bar{m}^l\\
=& (R_{ijkl} + 2K_{i[k}K_{l]j}) n^i \bar{m}^j n^k \bar{m}^l \\
&- 8 (K_{j[k,l]} + \Gamma^p_{j[k} K_{l]p}) n^{[0} \bar{m}^{j]} n^k \bar{m}^l \\
&+ 4 (R_{jl} - K_{jp} K^p_l + KK_{jl}) n^{[0} \bar{m}^{j]} n^{[0} \bar{m}^{l]}.
\end{align}
Note here the brackets around pairs of indices. This indicates the antisymmetric part of a tensor; that is for some arbitrary tensor $A_{ij}$, $A_{[ij]} = \frac{1}{2}(A_{ij}-A_{ji})$. This applies identically for indices belonging to separate tensors as well as superscripts in place of subscripts.

To make it easier to track the construction of this expression, we will break it down into three parts, by first defining each of the parenthetical terms above separately. We will also use the barred expressions. That is, let 
\begin{align}
\text{GaussDDDD} =& \bar{R}_{ijkl} + 2K_{i[k}K_{l]j} \\
\text{CodazziDDD} =& K_{j[k,l]} + \bar{\Gamma}^p_{j[k} K_{l]p} \\
\text{RojoDD} = & \bar{R}_{jl} - K_{jp} K^p_l + KK_{jl},
\end{align}
where these quantities are so named because of their relation to the Gauss-Codazzi equations. Then, we simply contract these with the tetrad we chose earlier to arrive at an expression for $\psi_4$. The barred Christoffel symbols and barred Ricci tensor have already been calculated by BSSN_RHSs.py, so we use those values.

In [35]:
# Step 5: Build the formula for \psi_4.
# Gauss equation: involving the Riemann tensor and extrinsic curvature.
GaussDDDD = ixp.zerorank4()
for i in range(DIM):
    for j in range(DIM):
        for k in range(DIM):
            for l in range(DIM):
                GaussDDDD[i][j][k][l] = RiemannbarDDDD[i][j][k][l] + extrinsicKDD[i][k]*extrinsicKDD[l][j] - extrinsicKDD[i][l]*extrinsicKDD[k][j]

# Codazzi equation: involving partial derivatives of the extrinsic curvature. 
# We will first need to declare derivatives of extrinsicKDD
extrinsicKDD_dD = ixp.declarerank3("extrinsicKDD_dD","sym12")
CodazziDDD = ixp.zerorank3()
for j in range(DIM):
    for k in range(DIM):
        for l in range(DIM):
            CodazziDDD[j][k][l] = extrinsicKDD_dD[j][l][k] - extrinsicKDD_dD[j][k][l]
            for p in range(DIM):
                CodazziDDD[j][k][l] += bssn.GammabarUDD[p][j][l]*extrinsicKDD[k][p] - bssn.GammabarUDD[p][j][k]*extrinsicKDD[l][p]

# Another piece. While not associated with any particular equation,
# this is still useful for organizational purposes.
RojoDD = ixp.zerorank2()
for j in range(DIM):
    for l in range(DIM):
        RojoDD[j][l] = bssn.trK*extrinsicKDD[j][l]
        for p in range(DIM):
            for d in range(DIM):
                RojoDD[j][l] += bssn.gammabarUU[p][d]*RiemannbarDDDD[j][p][l][d] \ 
                                - extrinsicKDD[j][p]*bssn.gammabarUU[p][d]*extrinsicKDD[d][l]

# Now we can calculate $\psi_4$ itself!
psi4 = sp.sympify(0)
for l in range(DIM):
    for j in range(DIM):
        psi4 += RojoDD[j][l] * nn * nn * mtetbarU[j] * mtetbarU[l]
        for k in range(DIM):
            psi4 += CodazziDDD[j][k][l] * ntetU[k] * nn * mtetbarU[j] * mtetbarU[l]
            for i in range(DIM):
                psi4 += GaussDDDD[i][j][k][l] * ntetU[i] * ntetU[k] * mtetbarU[j] * mtetbarU[l]

cse_output = sp.cse(RojoDD[0][0],sp.numbered_symbols("tmp"))
#for commonsubexpression in cse_output:
#    print("hello?",commonsubexpression)
for commonsubexpression in cse_output[0]:
    print((str(commonsubexpression[0])+" = "+str(commonsubexpression[1])+";").replace("**","^"))
for i,result in enumerate(cse_output[1]):
   print("RojoDD00 = "+str(result)+";")

tmp0 = hDD00 + 1;
tmp1 = aDD00 + trK*(hDD00/3 + 1/3);
tmp2 = hDD02*hDD12;
tmp3 = hDD01^2;
tmp4 = hDD22 + 1;
tmp5 = hDD02^2;
tmp6 = hDD11 + 1;
tmp7 = hDD12^2;
tmp8 = tmp0*tmp4;
tmp9 = 1/(2*hDD01*tmp2 - tmp0*tmp7 - tmp3*tmp4 - tmp5*tmp6 + tmp6*tmp8);
tmp10 = tmp9*(-hDD01*tmp4 + tmp2);
tmp11 = tmp9*(hDD01*hDD12 - hDD02*tmp6);
tmp12 = tmp9*(hDD01*hDD02 - hDD12*tmp0);
tmp13 = tmp9*(tmp4*tmp6 - tmp7);
tmp14 = tmp13/2;
tmp15 = tmp9*(-tmp5 + tmp8);
tmp16 = tmp15/2;
tmp17 = tmp9*(tmp0*tmp6 - tmp3);
tmp18 = tmp17/2;
tmp19 = cf^(-4);
tmp20 = trK/3;
tmp21 = aDD01 + hDD01*tmp20;
tmp22 = aDD02 + hDD02*tmp20;
tmp23 = 2*tmp1*tmp19;
tmp24 = tmp10/2;
tmp25 = -hDD_dD012 + hDD_dD021 + hDD_dD120;
tmp26 = tmp12/2;
tmp27 = hDD_dD001*tmp24 + hDD_dD110*tmp16 + tmp25*tmp26;
tmp28 = hDD01*tmp27;
tmp29 = tmp11/2;
tmp30 = hDD_dD001*tmp29 + hDD_dD110*tmp26 + tmp18*tmp25;
tmp31 = hDD02*tmp30;
tmp32 = hDD_dD001*tmp14 + hDD_dD110*tmp24 + tmp25*tmp29;
tmp33 = tmp0*tmp32;
tmp34 = tmp28 + tmp31 + tmp33;
tmp35 = hDD_dD012